# Schema: XML Schema

Ähnlich wie die Document Type Definition (DTD), ermöglicht auch XML Schema die Spezifikation von XML Dokumente und somit das Validieren solcher Dokumente. Ziel beider Grammatiken ist die Interoperabilität zwischen entwickelten Systeme zu ermöglichen oder erhöhen. In dieser Übung schauen wir uns XML Schema etwas genauer in der Praxis an. Führen Sie zuerst den folgenden Codeblock aus und machen Sie dann der Reihe nach weiter. Beantworten Sie die Fragen (falls zutreffend). Zum Schluss, schreiben Sie ein eigenes XML Schema und ein exemplarisches XML Dokument dafür. Stellen Sie sicher, dass das XML Dokument wohlgeformt und gültig ist.

In [1]:
import io
from lxml import etree as et

def isvalid(xsd, doc):
    print(et.XMLSchema(et.parse(io.StringIO(xsd))).validate(et.fromstring(doc)))

In [3]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="discography"/>
</xs:schema>
"""

isvalid(xsd, '<discography/>')
isvalid(xsd, '<albums/>') # Warum ist dieses Dokument nicht gültig? Antwort: Das Schema deklariert kein albums Element.

True
False


In [5]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="title" type="xs:string"/>
</xs:schema>
"""

isvalid(xsd, '<title>The Dark Side of the Moon</title>')
isvalid(xsd, '<title>1973</title>') # Warum ist dieses Dokument gültig? Antwort: Das Schema deklariert das Element title mit Datentyp string (Zeichenfolge). Die (eigentliche) Nummer 1973 wird hier als Zeichenfolge interpretiert.

True
True


In [7]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="year" type="xs:decimal"/>
</xs:schema>
"""

isvalid(xsd, '<year>1973</year>')
isvalid(xsd, '<year>The Dark Side of the Moon</year>') # Warum ist dieses Dokument nicht gültig? Antwort: Der Inhalt des Elements year ist als Dezimalzahl deklariert. Eine Zeichenfolge kann nicht als Zahl interpretiert werden.

True
False


In [8]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="year" type="xs:gYear"/>
</xs:schema>
"""

isvalid(xsd, '<year>1973</year>')

True


In [13]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="year" type="xs:gMonth"/>
</xs:schema>
"""

isvalid(xsd, '<year>1979</year>') # Warum ist dieses Dokument nicht gültig? Antwort: Der Datentyp des Elements year ist gMonth und 1973 ist ein Wert der nicht in der Wertemenge dieses Datentyps liegt. Gültig wäre z.B. <year>--07</year>.

False


## Komplexe Elemente Deklarieren: Variante I

In [14]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="title" type="xs:string"/>
        <xs:element name="label" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

True


In [16]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="label" type="xs:string"/>
        <xs:element name="title" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: Das Schema erwartet die Reihenfolge label, titel für diese Elemente.

False


In [18]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="title" type="xs:string"/>
        <xs:element name="label" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<label>Harvest, EMI</label>
<title>The Dark Side of the Moon</title>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: Das Schema erwartet die Reihenfolge title, label für diese Elemente.

False


## Komplexe Elemente Deklarieren: Variante II

In [19]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album" type="AlbumType"/>
  
  <xs:complexType name="AlbumType">
    <xs:sequence>
      <xs:element name="title" type="xs:string"/>
      <xs:element name="label" type="xs:string"/>
    </xs:sequence>
  </xs:complexType>

</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

True


In [21]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="work" type="WorkType"/>
  <xs:element name="album" type="AlbumType"/>
  <xs:complexType name="WorkType">
    <xs:sequence>
      <xs:element name="title" type="xs:string"/>
    </xs:sequence>
  </xs:complexType>
  <xs:complexType name="AlbumType">
    <xs:complexContent>
      <xs:extension base="WorkType">
        <xs:sequence>
          <xs:element name="label" type="xs:string"/>
        </xs:sequence>
      </xs:extension>
    </xs:complexContent>
 </xs:complexType>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

doc = """
<album>
<!-- The order matters -->
<label>Harvest, EMI</label>
<title>The Dark Side of the Moon</title>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: Das Schema erwartet die Reihenfolge title, label für diese Element. Der Type AlbumType erweitert den Typ WorkType der zuerst das Element title deklariert. Danach deklariert AlbumType das Element label.

True
False


# Attribute Deklarieren

In [22]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:attribute name="title" type="xs:string"/>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album title="The Dark Side of the Moon"/>
"""

isvalid(xsd, doc)

True


In [23]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:simpleContent>
        <xs:extension base="xs:string">
          <xs:attribute name="year" type="xs:gYear" />
        </xs:extension>
      </xs:simpleContent>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album year="1973">The Dark Side of the Moon</album>
"""

isvalid(xsd, doc)

True


# Weitere Eigenschaften

In [25]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="title" type="xs:string" maxOccurs="2"/>
        <xs:element name="label" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<title>The Bright Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument gültig? Antwort: Das Element title darf ein oder zweimal verwendet werden.

True


In [27]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:all>
        <xs:element name="title" type="xs:string"/>
        <xs:element name="label" type="xs:string"/>
      </xs:all>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

doc = """
<album>
<label>Harvest, EMI</label>
<title>The Dark Side of the Moon</title>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument gültig? Antwort: Der komplexe Typ deklariert hier die Elemente title, label nicht als Sequenz sondern ohne definierte Reihenfolge.

True
True


In [29]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:choice>
        <xs:element name="title" type="xs:string"/>
        <xs:element name="label" type="xs:string"/>
      </xs:choice>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
</album>
"""

isvalid(xsd, doc)

doc = """
<album>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: Das Schema spezifiziert hier, dass das Element title oder das Element label verwendet werden kann, nicht aber beide.

True
True
False


In [36]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="released">
    <xs:simpleType>
      <xs:restriction base="xs:integer">
        <xs:minInclusive value="1950"/>
        <xs:maxInclusive value="2018"/>
      </xs:restriction>
    </xs:simpleType>
  </xs:element>
</xs:schema>
"""

doc = """
<released>1973</released>
"""

isvalid(xsd, doc)

doc = """
<released>2020</released>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: Der Wert 2020 liegt ausserhalb des Bereiches [1950, 2018].

True
False


In [37]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="author">
    <xs:simpleType>
      <xs:restriction base="xs:string">
        <xs:enumeration value="Roger Waters"/>
        <xs:enumeration value="David Gilmour"/>
        <xs:enumeration value="Syd Barrett"/>
      </xs:restriction>
    </xs:simpleType>
  </xs:element>
</xs:schema>
"""

doc = """
<author>Roger Waters</author>
"""

isvalid(xsd, doc)

doc = """
<author>Jimmy Page</author>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: Der Wert Jimmy Page ist nicht in der Menge {'Roger Waters', 'David Gilmour', 'Syd Barret'}.

True
False


In [39]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="author">
    <xs:simpleType>
      <xs:restriction base="xs:string">
        <xs:pattern value="[A-Z][a-z]+"/>
      </xs:restriction>
    </xs:simpleType>
  </xs:element>
</xs:schema>
"""

doc = """
<author>Waters</author>
"""

isvalid(xsd, doc)

doc = """
<author>gilmour</author>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: Der Wert 'gilmour' entspricht nicht dem Pattern: Müsste Gilmour lauten.

True
False


In [40]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema" 
  targetNamespace="http://example.org"
  elementFormDefault="qualified">
  <xs:element name="album">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="title" type="xs:string"/>
        <xs:element name="label" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album xmlns="http://example.org">
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

True


Denken Sie sich nun ein eigenes XML Dokument aus und erstellen Sie dafür ein XML Schema. Überprüfen Sie das XML Dokument auf Gültigkeit.

In [44]:
dtd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="addresses">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="address" maxOccurs="unbounded">
          <xs:complexType>
            <xs:sequence>
              <xs:element name="streetname" type="xs:string"/>
              <xs:element name="streetnumber" type="xs:string"/>
              <xs:element name="zip" type="xs:int" minOccurs="0"/>
              <xs:element name="city" type="xs:string"/>
            </xs:sequence>
          </xs:complexType>
        </xs:element>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<addresses>
<address>
<streetname>Lange Laube</streetname>
<streetnumber>43</streetnumber>
<zip>30178</zip>
<city>Hannover</city>
</address>
<address>
<streetname>Tintentrift</streetname>
<streetnumber>4b</streetnumber>
<city>Hannover</city>
</address>
</addresses>
"""

isvalid(dtd, doc)

True
